# Machine Learning Engineer Nanodegree
## Capstone Project
Simon Jackson
March 21st, 2017

---

In [1]:
import os
import zipfile
import numpy as np
import pandas as pd
from IPython.display import display # Allows the use of display() for DataFrames

from scripts.utility import help_read_csv

## Data

Data was downloaded as .zip format...

Extract all data from zip files into the same directory

In [2]:
# Utility variables
data_directory = "data"
folders_to_unzip = ["imdb-5000-movie-dataset.zip", "movielens-20m-dataset.zip"]

In [3]:
# Extract all files from zip folders in data directory,
# into data directory
if False:
    for zip_file in folders_to_unzip:
        print "Extracting all from " + zip_file + "...\t",
        path_to_zip_file = os.path.join(data_directory, zip_file)
        zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
        zip_ref.extractall(data_directory)
        zip_ref.close()
        print "DONE"
else:
    print "Extraction turned off. Check that files have been extracted."

Extraction turned off. Check that files have been extracted.


In [4]:
print "Files in the data directory:\n"
for f in os.listdir(data_directory):
    print "- " + f

Files in the data directory:

- genome_scores.csv
- genome_tags.csv
- imdb-5000-movie-dataset.zip
- link.csv
- movie.csv
- movielens-20m-dataset.zip
- movie_metadata.csv
- rating.csv
- tag.csv


### IMDB data

In [45]:
imdb = help_read_csv(os.path.join(data_directory, 'movie_metadata.csv'))

data\movie_metadata.csv read with 5043 rows and 28 columns.


In [18]:
# Column types
imdb.dtypes

color                         object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
gross                        float64
genres                        object
actor_1_name                  object
movie_title                   object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
plot_keywords                 object
movie_imdb_link               object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
aspect_ratio                 float64
m

In [13]:
# Examine few rows
imdb.head(n = 5)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


This information will be used for content-based filtering of movies. Certain variables are not useful and can be dropped. These include:

- Variables holding names (`director_name`, `actor_1_name`, `actor_2_name`, `actor_3_name`).
    - There are too many to worry about.
- `plot_keywords`
    - These are often very movie-specific and almost all are unique.
- `movie_title` and `movie_imdb_link`
    - All unique, BUT useful for matching to other data sets later

In [46]:
# Separate title and IMDB link
imdb_titles = imdb[['movie_title', 'movie_imdb_link']]

imdb.drop(['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name',
           'plot_keywords', 'movie_title', 'movie_imdb_link'], axis = 1, inplace = True)

print 'Titles data has {} rows and {} columns'.format(*imdb_titles.shape) 
print 'Movie data has {} rows and {} columns'.format(*imdb.shape) 

Titles data has 5043 rows and 2 columns
Movie data has 5043 rows and 21 columns


In [35]:
imdb['content_rating'].value_counts()
#imdb['content_rating']#.nunique

R            2118
PG-13        1461
PG            701
Not Rated     116
G             112
Unrated        62
Approved       55
TV-14          30
TV-MA          20
X              13
TV-PG          13
TV-G           10
Passed          9
NC-17           7
GP              6
M               5
TV-Y7           1
TV-Y            1
Name: content_rating, dtype: int64

#### Categorical variables

- `language`: convert to 1 = English, 0 = other
- `genres`: will need to be split into multiple variables indicating whether certain genres were tagged.
- `country`: 1 = USA, 0 = Other (possibly a separate one for UK?)
- `content_rating`: will need a number of categories (R, PG, PG-13, G, etc)